## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Extract Social Determinants of Health](https://aws.amazon.com/marketplace/pp/prodview-4nvvaaklzddsy)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Background

- **Model**: `en.med_ner.sdoh_langtest.pipeline`

- **Model Description**: It is designed to detect and label social determinants of health (SDOH) entities within text data. Social determinants of health are crucial factors that influence individuals’ health outcomes, encompassing various social, economic, and environmental element. It has been trained using advanced machine learning techniques on a diverse range of text sources. It can accurately recognize and classify a wide range of SDOH entities, including but not limited to factors such as socioeconomic status, education level, housing conditions, access to healthcare services, employment status, cultural and ethnic background, neighborhood characteristics, and environmental factors. The model’s accuracy and precision have been carefully validated against expert-labeled data to ensure reliable and consistent results.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "en-med-ner-sdoh-langtest-pipeline"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"


### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-------!

Once endpoint has been created, you would be able to perform real-time inference.

In [6]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [7]:
docs = [
    "Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",
    "Jane works as a cleaning assistant and cannot access health insurance or paid sick leave. She has a son, a student at college. Pt with likely long-standing depression. She is aware she needs rehab. Pt reports having her catholic faith as a means of support as well.  She has a long history of etoh abuse, beginning in her teens. She reports she has been a daily drinker for 30 years, most recently drinking beer daily. She smokes a pack of cigarettes a day. She had DUI in April and was due to court this week."

]


sample_text = """Kate a cleaning assistant without health insurance or paid sick leave, has struggled with alcohol abuse since her teens, drinking beer daily for 30 years and smoking a pack of cigarettes daily. With a DUI in April and a recent court date, she leans on her Catholic faith for support, alongside managing depression and considering rehab."""

### JSON

#### Example 1

  **Input format**:
  
  
```json
{
    "text": "Single text document"
}
```

In [8]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"])

,document,ner_chunk,begin,end,ner_label,ner_confidence
0,"Kate a cleaning assistant without health insurance or paid sick leave, has struggled with alcohol abuse since her teens, drinking beer daily for 30 years and smoking a pack of cigarettes daily. With a DUI in April and a recent court date, she leans on her Catholic faith for support, alongside managing depression and considering rehab.",cleaning assistant,7,24,Employment,0.57879996
1,"Kate a cleaning assistant without health insurance or paid sick leave, has struggled with alcohol abuse since her teens, drinking beer daily for 30 years and smoking a pack of cigarettes daily. With a DUI in April and a recent court date, she leans on her Catholic faith for support, alongside managing depression and considering rehab.",health insurance,34,49,Insurance_Status,0.57885003
2,"Kate a cleaning assistant without health insurance or paid sick leave, has struggled with alcohol abuse since her teens, drinking beer daily for 30 years and smoking a pack of cigarettes daily. With a DUI in April and a recent court date, she leans on her Catholic faith for support, alongside managing depression and considering rehab.",alcohol abuse,90,102,Alcohol,0.8004
3,"Kate a cleaning assistant without health insurance or paid sick leave, has struggled with alcohol abuse since her teens, drinking beer daily for 30 years and smoking a pack of cigarettes daily. With a DUI in April and a recent court date, she leans on her Catholic faith for support, alongside managing depression and considering rehab.",her,110,112,Gender,0.9608
4,"Kate a cleaning assistant without health insurance or paid sick leave, has struggled with alcohol abuse since her teens, drinking beer daily for 30 years and smoking a pack of cigarettes daily. With a DUI in April and a recent court date, she leans on her Catholic faith for support, alongside managing depression and considering rehab.",teens,114,118,Age,0.5261
5,"Kate a cleaning assistant without health insurance or paid sick leave, has struggled with alcohol abuse since her teens, drinking beer daily for 30 years and smoking a pack of cigarettes daily. With a DUI in April and a recent court date, she leans on her Catholic faith for support, alongside managing depression and considering rehab.",drinking,121,128,Alcohol,0.9951
6,"Kate a cleaning assistant without health insurance or paid sick leave, has struggled with alcohol abuse since her teens, drinking beer daily for 30 years and smoking a pack of cigarettes daily. With a DUI in April and a recent court date, she leans on her Catholic faith for support, alongside managing depression and considering rehab.",beer,130,133,Alcohol,0.9877
7,"Kate a cleaning assistant without health insurance or paid sick leave, has struggled with alcohol abuse since her teens, drinking beer daily for 30 years and smoking a pack of cigarettes daily. With a DUI in April and a recent court date, she leans on her Catholic faith for support, alongside managing depression and considering rehab.",daily,135,139,Substance_Frequency,0.9977
8,"Kate a cleaning assistant without health insurance or paid sick leave, has struggled with alcohol abuse since her teens, drinking beer daily for 30 years and smoking a pack of cigarettes daily. With a DUI in April and a recent court date, she leans on her Catholic faith for support, alongside managing depression and considering rehab.",30 years,145,152,Substance_Duration,0.75750005
9,"Kate a cleaning assistant without health insurance or paid sick leave, has struggled with alcohol abuse since her teens, drinking beer daily for 30 years and smoking a pack of cigarettes daily. With a DUI in April and a recent court date, she leans on her Catholic faith for support, alongside managing depression and considering rehab.",smoking,158,164,Smoking,0.997


#### Example 2

  **Input format**:
  
  
```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

In [9]:
input_json_data = {"text": docs}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"])

,document,ner_chunk,begin,end,ner_label,ner_confidence
0,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",55 years old,9,20,Age,0.98226666
1,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",New York,33,40,Geographic_Entity,0.7573
2,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",divorced,45,52,Marital_Status,0.9999
3,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",Mexcian American,54,69,Race_Ethnicity,0.42835
4,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",woman,71,75,Gender,0.9906
5,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",financial problems,82,99,Financial_Status,0.66255003
6,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",She,102,104,Gender,0.9987
7,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",Spanish,113,119,Language,0.9997
8,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",Portuguese,125,134,Language,0.9982
9,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",She,137,139,Gender,0.9975


### JSON Lines

  **Input format**:
  
```json
{"text": "Text document 1"}
{"text": "Text document 2"}
```

In [10]:
import json

def create_jsonl(records):
    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

input_jsonl_data = create_jsonl(docs)

#### Example 1

In [11]:
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"ner_chunk": "55 years old", "begin": 9, "end": 20, "ner_label": "Age", "ner_confidence": "0.98226666"}, {"ner_chunk": "New York", "begin": 33, "end": 40, "ner_label": "Geographic_Entity", "ner_confidence": "0.7573"}, {"ner_chunk": "divorced", "begin": 45, "end": 52, "ner_label": "Marital_Status", "ner_confidence": "0.9999"}, {"ner_chunk": "Mexcian American", "begin": 54, "end": 69, "ner_label": "Race_Ethnicity", "ner_confidence": "0.42835"}, {"ner_chunk": "woman", "begin": 71, "end": 75, "ner_label": "Gender", "ner_confidence": "0.9906"}, {"ner_chunk": "financial problems", "begin": 82, "end": 99, "ner_label": "Financial_Status", "ner_confidence": "0.66255003"}, {"ner_chunk": "She", "begin": 102, "end": 104, "ner_label": "Gender", "ner_confidence": "0.9987"}, {"ner_chunk": "Spanish", "begin": 113, "end": 119, "ner_label": "Language", "ner_confidence": "0.9997"}, {"ner_chunk": "Portuguese", "begin": 125, "end": 134, "ner_label": "Language", "ner_confidence": "0.9982"}

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [12]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [13]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [14]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [16]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data["predictions"])
    display(df)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [17]:
process_s3_json_output_and_save(validation_json_file_name)

,document,ner_chunk,begin,end,ner_label,ner_confidence
0,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",55 years old,9,20,Age,0.98226666
1,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",New York,33,40,Geographic_Entity,0.7573
2,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",divorced,45,52,Marital_Status,0.9999
3,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",Mexcian American,54,69,Race_Ethnicity,0.42835
4,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",woman,71,75,Gender,0.9906
5,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",financial problems,82,99,Financial_Status,0.66255003
6,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",She,102,104,Gender,0.9987
7,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",Spanish,113,119,Language,0.9997
8,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",Portuguese,125,134,Language,0.9982
9,"Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels.",She,137,139,Gender,0.9975


### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [19]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [20]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"predictions": [{"ner_chunk": "55 years old", "begin": 9, "end": 20, "ner_label": "Age", "ner_confidence": "0.98226666"}, {"ner_chunk": "New York", "begin": 33, "end": 40, "ner_label": "Geographic_Entity", "ner_confidence": "0.7573"}, {"ner_chunk": "divorced", "begin": 45, "end": 52, "ner_label": "Marital_Status", "ner_confidence": "0.9999"}, {"ner_chunk": "Mexcian American", "begin": 54, "end": 69, "ner_label": "Race_Ethnicity", "ner_confidence": "0.42835"}, {"ner_chunk": "woman", "begin": 71, "end": 75, "ner_label": "Gender", "ner_confidence": "0.9906"}, {"ner_chunk": "financial problems", "begin": 82, "end": 99, "ner_label": "Financial_Status", "ner_confidence": "0.66255003"}, {"ner_chunk": "She", "begin": 102, "end": 104, "ner_label": "Gender", "ner_confidence": "0.9987"}, {"ner_chunk": "Spanish", "begin": 113, "end": 119, "ner_label": "Language", "ner_confidence": "0.9997"}, {"ner_chunk": "Portuguese", "begin": 125, "end": 134, "ner_label": "Language", "ner_confidence": "0.9982"}

In [21]:
model.delete_model()

INFO:sagemaker:Deleting model with name: en-med-ner-sdoh-langtest-pipeline-2024-05-07-09-33-12-367


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

